In [59]:
from bs4 import BeautifulSoup
import requests
import json
import time
import pandas as pd
import os

In [60]:
match_names = ['Final', 'Third-place', 'Semi-finals-1', 'Semi-finals-2', 'Quarter-final-1', 'Quarter-final-2', 
               'Quarter-final-3', 'Quarter-final-4', 'Round16-1', 'Round16-2', 'Round16-3', 
               'Round16-4', 'Round16-5', 'Round16-6', 'Round16-7', 'Round16-8']

In [79]:
def get_names(match, url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find_all('table', class_ = 'stats_table sortable')
    table_kp = soup.find_all('div', class_ = 'table_wrapper')
    keys = [i.select('caption')[0].text for i in table]
    keys_n = [keys[0], keys[7]]
    tags = ['summary', 'passing', 'pass types', 'defensive actions', 'possession', 'miscellaneous stats', 
            'summary', 'passing', 'pass types', 'defensive actions', 'possession', 'miscellaneous stats']
    for i in range(12):
        keys[i] = keys[i].split()[0] + '-' + tags[i]
        match[keys[i]] = pd.read_html(str(table[i]))[0]
    for i in range(2):
        match[keys_n[i].split()[0]+' keeper_stats'] = pd.read_html(str(table_kp[2 * i+1]))[0]

In [62]:
def get_match(event, event_stats, url):
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    matches = soup.find_all('div', class_ = 'match-summary')

    for i in range(len(match_names)):

        match = match_names[i]
        event[match]={}
        event_stats[match]={}

        event[match]['winner'] = matches[i].select('a')[0].text
        event[match]['loser'] = matches[i].select('a')[2].text
        event[match]['winner_goal'] = matches[i].select('a')[1].text.split('–')[0]
        event[match]['loser_goal'] = matches[i].select('a')[1].text.split('–')[0]
        url_match = 'https://fbref.com' + matches[i].select('a')[1].get('href')

        get_names(event_stats[match], url_match)
        
        #add delay to void overloading
        time.sleep(4)

In [63]:
WC_2019 = {}
WC_2019_stats = {}

In [80]:
get_match(WC_2019, WC_2019_stats, 'https://fbref.com/en/comps/106/2019/2019-Womens-World-Cup-Stats')

In [82]:
for key in match_names:
    os.mkdir('WC_2019/'+key)

In [83]:
for key in WC_2019.keys():
    df = pd.DataFrame.from_dict(WC_2019[key],orient='index')
    df.to_csv('WC_2019/'+key+'/result.csv', index = False)

In [84]:
for key in WC_2019.keys():
    for tag in WC_2019_stats[key].keys():
        df = WC_2019_stats[key][tag]
        df.to_csv('WC_2019/'+key+'/'+tag+'.csv', index = False)